# Notebook version of 2.3_dps.py, which runs over a single tile
#### cant get 2.3_dps.py to work, so alternate approach is to loop over tile ids and return indiv tile CSVs of ATL08

In [1]:
#import pdal
import json
import os
import glob

import itertools

import pandas as pd
from pyproj import CRS, Transformer

# From CovariateUtils
import numpy
from rasterio import enums
from rasterio.io import MemoryFile
from rasterio.crs import CRS
from rasterio.vrt import WarpedVRT
from rasterio.warp import array_bounds, calculate_default_transform
import geopandas

import argparse

from maap.maap import MAAP
maap = MAAP()

import sys
if False:
    sys.path.append('/projects/code/icesat2_boreal/notebooks/3.Gridded_product_development')
    sys.path.append('/projects/code/icesat2_boreal/notebooks/2.ICESat-2_processing')

    #TODO: how to get this import right if its in a different dir
    from CovariateUtils import get_index_tile
    
import FilterUtils
import ExtractUtils
import CovariateUtils


/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
maap_query = True
dps_dir = '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17' #'/projects/jabba/dps_output/2.3_output'#
output_dir = '/projects/jabba/data/out_tiles'
csv_list_fn = '/projects/jabba/data/extract_atl08_csv_list.csv'
TEST = False
do_30m = True
extract_covars = False

in_tile_fn = '/projects/maap-users/alexdevseed/boreal_tiles.gpkg'
in_tile_layer = 'boreal_tiles_albers'

thresh_h_can = 100
thresh_h_dif = 100
month_min = 6
month_max = 9

date_start = '06-01'
date_end = '09-30'

# NA tiles
# Read the boreal tile index file

boreal_tile_index = geopandas.read_file(in_tile_fn)
boreal_tile_index_subset = boreal_tile_index.to_crs(4326).cx[-170:-50, 50:75]

# Boreal NA tiles: need just a list of tile_ids
INPUT_TILE_NUM_LIST = boreal_tile_index_subset['layer'].astype(int).tolist()

INPUT_TILE_NUM_LIST[0:5]

if False:
    # TODO: Should not do glob.glob by tile
    # Get a list of all ATL08 CSV files from (from extract_atl08) (this will be a large boreal list)
    print("\tDPS dir to find ATL08 CSVs: {}".format(dps_dir))
    all_atl08_csvs = glob.glob(dps_dir + "/**/ATL08*.csv", recursive=True)


#### Build the list of extracted CSV paths from DPS and save all paths in a list - glob.glob takes a while - dont do this for each tile

In [3]:
%%time
# This is sloooow
if False:
    print(dps_dir + "/**/ATL08*" + seg_str + ".csv") 
    seg_str="_30m"
    all_atl08_csvs = glob.glob(dps_dir + "/**/ATL08*" + seg_str + ".csv", recursive=True)
    print(len(all_atl08_csvs))
    all_atl08_csvs_df = pd.DataFrame({"path": all_atl08_csvs})
    all_atl08_csvs_df.to_csv(csv_list_fn)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.77 µs


In [4]:
def find_atl08_csv_tile_BACKUP(all_atl08_for_tile, all_atl08_csvs_df, seg_str):
    
    print("seg_str: ", seg_str)
    print("\tFind ATL08 CSVs for tile...")
    # Change the small ATL08 H5 granule names to match the output filenames from extract_atl08.py (eg, ATL08_*_30m.csv)
    all_atl08_for_tile_CSVname = [os.path.basename(f).replace("ATL08", "ATL08"+seg_str).replace('.h5', seg_str+'.csv') for f in all_atl08_for_tile]

    print('\t\tLength of all ATL08 for tile: {}'.format(len(all_atl08_for_tile)))
    all_atl08_csvs = all_atl08_csvs_df['path'].to_list()
    print('\t\tLength of all_atl08_csvs: {}'.format(len(all_atl08_csvs)))
    
    # Get basenames of CSVs
    all_atl08_csvs_BASENAME = [os.path.basename(f) for f in all_atl08_csvs]
    
    #print(all_atl08_for_tile_CSVname)
    # Get index of ATL08 in tile bounds from the large list of all ATL08 CSVs
    names = [name for i, name in enumerate(all_atl08_for_tile_CSVname) if name in set(all_atl08_csvs_BASENAME)]
    print(names)
    idx = [all_atl08_csvs_BASENAME.index(name) for name in names]
    print(idx)
    print('\t\tLength of idx with matches between ATL08 CSVs and ATL08 granules for tile: {}'.format(len(idx)))
    all_atl08_csvs_FOUND  = [all_atl08_csvs[i] for i in idx]
    print(all_atl08_csvs_FOUND)
    ## Get the subset of all ATL08 CSVs that just correspond to the ATL08 H5 intersecting the current tile
    #all_atl08_h5_with_csvs_for_tile = [all_atl08_for_tile[x] for x in idx]       
    #print(all_atl08_h5_with_csvs_for_tile)

    if False:
        # Check to make sure these are in fact files (necessary?)
        all_atl08_csvs_NOT_FOUND = []
        all_atl08_csvs_FOUND = []
        for file in all_atl08_h5_with_csvs_for_tile:
            print("seg_str: ", seg_str)
            # Convert the h5 file string back to the actual string of the CSV file
            csv_fn = os.path.basename(file).replace('.h5',seg_str+'.csv').replace("ATL08_","ATL08"+seg_str+"_")
            print("\t\tcsv_fn: ", csv_fn)
            # Find this CSV path from large list
            name = [name for i, name in enumerate(all_atl08_csvs) if name in csv_fn]
            print("\t\tname: ",name)
            file = os.path.join(dps_dir, csv_fn)    
            print(file)
            if not os.path.isfile(file):
                all_atl08_csvs_NOT_FOUND.append(file)
            else:
                all_atl08_csvs_FOUND.append(file)
            
    return(all_atl08_csvs_FOUND)

# Loop over input tiles to output ATL08 tile CSVs

In [ ]:
%%time

#for in_tile_num in INPUT_TILE_NUM_LIST[100:105]:
for in_tile_num in [30542]:#, 30543, 30821, 30822, 30823]:
    
    # TODO: make this an arg
    years_list = [2019, 2020, 2021]
    
    seg_str = '_100m'
    if do_30m:
        seg_str = '_30m'
    if TEST:
        seg_str = ''
    
    
    
    if maap_query and dps_dir is not None:
        
        print("\nDoing MAAP query by tile bounds to find all intersecting ATL08 ")
        # Get a list of all ATL08 H5 granule names intersecting the tile (this will be a small list)
        # all_atl08_for_tile = ExtractUtils.get_h5_list() #<- when you get import to work, change back to this
        all_atl08_for_tile = ExtractUtils.get_h5_list(tile_num=in_tile_num, tile_fn=in_tile_fn, layer=in_tile_layer, DATE_START=date_start, DATE_END=date_end, YEARS=years_list)
              
        if not os.path.isfile(csv_list_fn):
            # Get a list of all ATL08 CSV files from (from extract_atl08) (this will be a large boreal list)
            print("\tDPS dir to find ATL08 CSVs: {}".format(dps_dir))
            all_atl08_csvs = glob.glob(dps_dir + "/**/ATL08*" + seg_str + ".csv", recursive=True)
            print(len(all_atl08_csvs))
            all_atl08_csvs_df = pd.DataFrame({"path": all_atl08_csvs})
            all_atl08_csvs_df.to_csv(csv_list_fn)
        else:
            print("\tReading existing list og ATL08 CSVs: {}".format(csv_list_fn))
            all_atl08_csvs_df = pd.read_csv(csv_list_fn)
            
        # Find the ATL08 CSVs from extract that are associated with the ATL08 granules that intersect this tile
        # These CSvs are nested deep after DPS runs
        # They should match all the ATL08 granules, but probably wont bc: (1) did DPS for ATL08 30m fully complete with no fails? (2) did DPS for extract fully complete with no fails?
        all_atl08_csvs_FOUND = FilterUtils.find_atl08_csv_tile(all_atl08_for_tile, all_atl08_csvs_df, seg_str) 
        
        #all_atl08_csvs_FOUND = [x for x in all_atl08_h5_with_csvs_for_tile if x not in all_atl08_csvs_NOT_FOUND]
        print("\t# of ATL08 CSV found for tile {}: {}".format(in_tile_num, len(all_atl08_csvs_FOUND)))
        if len(all_atl08_csvs_FOUND) == 0:
            print('\tNo ATL08 extracted for this tile.')
            continue
        
        # Merge all ATL08 CSV files for the current tile into a pandas df
        print("Creating pandas data frame...")
        atl08 = pd.concat([pd.read_csv(f) for f in all_atl08_csvs_FOUND ], sort=False, ignore_index=True)
        #print(atl08_tmp.info())
        print("\nFiltering by tile: {}".format(in_tile_num))
        if False:
            # Get tile bounds as xmin,xmax,ymin,ymax
            print('DEBUG reorder_4326_bounds')
            in_bounds = FilterUtils.reorder_4326_bounds(in_tile_fn, in_tile_num, buffer=0, layer=in_tile_layer)
            print(in_bounds)
        
        if False:
            # Now filter ATL08 obs by tile bounds
            print('DEBUG filter_atl08_bound')
            atl08 = FilterUtils.filter_atl08_bounds(atl08_df=atl08, in_bounds=in_bounds)

    elif maap_query and dps_dir is None:
        print("\nNo DPS dir specified: cant get ATL08 CSV list to match with tile bound results from MAAP query.\n")
        os._exit(1)
    else:
        # Filter by bounds: EPT with a the bounds from an input tile
        atl08 = FilterUtils.filter_atl08_bounds_tile_ept(in_ept_fn, in_tile_fn, in_tile_num, in_tile_layer, output_dir, return_pdf=True)
    
    ## Filter by quality: based on a standard filter_atl08_qual() function that we use across all notebooks, scripts, etc
    #atl08_pdf_filt = FilterUtils.filter_atl08_qual(atl08, out_cols_list)
    # Filter by quality
    print('DEBUG filter_atl08_qual')
    atl08_pdf_filt = FilterUtils.filter_atl08_qual(atl08, SUBSET_COLS=True, 
                                                       subset_cols_list=['rh25','rh50','rh60','rh70','rh75','rh80','rh85','rh90','rh95','h_can','h_max_can'], 
                                                       filt_cols=['h_can','h_dif_ref','m','msw_flg','beam_type','seg_snow', 'seg_landcov'], 
                                                       thresh_h_can=100, thresh_h_dif=100, month_min=6, month_max=9)
    atl08=None
    
    # Convert to geopandas data frame in lat/lon
    atl08_gdf = GeoDataFrame(atl08_pdf_filt, geometry=geopandas.points_from_xy(atl08_pdf_filt.lon, atl08_pdf_filt.lat), crs='epsg:4326')
    out_name_stem = "atl08_filt"
    atl08_pdf_filt=None
    
    if extract_covars:
        ### Below here should be re-worked to follow final chunk of nb 2.3 (6/15/2021)
        #
        
        # Extract topo covar values to ATL08 obs (doing a reproject to tile crs)
        # TODO: consider just running 3.1.5_dpy.py here to produce this topo stack right before extracting its values
        topo_covar_fn = do_3_1_5_dp.main(in_tile_fn=in_tile_fn, in_tile_num=in_tile_num, tile_buffer_m=120, in_tile_layer=in_tile_layer, topo_tile_fn='https://maap-ops-dataset.s3.amazonaws.com/maap-users/alexdevseed/dem30m_tiles.geojson')
        atl08_gdf_out = ExtractUtils.extract_value_gdf(topo_covar_fn, atl08_gdf, ["elevation","slope","tsri","tpi", "slopemask"], reproject=True)
        out_name_stem = out_name_stem + "_topo"

        # Extract landsat covar values to ATL08 obs
        # TODO: consider just running 3.1.2_dpy.py here
        landsat_covar_fn = do_3_1_2_dps.main(in_tile_fn=in_tile_fn, in_tile_num=in_tile_num, in_tile_layer=in_tile_layer, sat_api='https://landsatlook.usgs.gov/sat-api', local=args.local)
        atl08_gdf_out = ExtractUtils.extract_value_gdf(landsat_covar_fn, atl08_gdf_out, ['Blue', 'Green', 'Red', 'NIR', 'SWIR', 'NDVI', 'SAVI', 'MSAVI', 'NDMI', 'EVI', 'NBR', 'NBR2', 'TCB', 'TCG', 'TCW', 'ValidMask', 'Xgeo', 'Ygeo'], reproject=False)
        out_name_stem = out_name_stem + "_landsat"
        
    # CSV the file
    cur_time = time.strftime("%Y%m%d%H%M%S")
    out_csv_fn = os.path.join(output_dir, out_name_stem + "_" + cur_time + ".csv")
    atl08_gdf_out.to_csv(out_csv_fn,index=False, encoding="utf-8-sig")
    
    print("Wrote output csv of filtered ATL08 obs with topo and Landsat covariates for tile {}: {}".format(in_tile_num, out_csv_fn) )


Doing MAAP query by tile bounds to find all intersecting ATL08 
Getting tile polygon index...
	TILE_NUM: 30542 (-117.10749852280769,50.78795362739066,-116.50936927974429,51.16389512140189)
		# ATL08 for tile 30542: 23
	Reading existing list og ATL08 CSVs: /projects/jabba/data/extract_atl08_csv_list.csv
seg_str:  _30m
	Find ATL08 CSVs for tile...
		Length of all ATL08 for tile: 23
		Length of all_atl08_csvs: 10513
['ATL08_30m_20190621110851_12860302_003_01_30m.csv', 'ATL08_30m_20190818082057_07830402_003_01_30m.csv', 'ATL08_30m_20190822081238_08440402_003_01_30m.csv']
[8766, 9875, 9768]
		Length of idx with matches between ATL08 CSVs and ATL08 granules for tile: 3
['/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/22/56/24/607120/ATL08_30m_20190621110851_12860302_003_01_30m.csv', '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/12/32/945664/ATL08_30m_20190818082057_07830402_003_01_30m.csv', '/projects/r2d2/dps_output/run_extract_atl

In [ ]:
import pandas as pd
all_atl08_csvs_FOUND = ['/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/22/56/24/607120/ATL08_30m_20190621110851_12860302_003_01_30m.csv', '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/12/32/945664/ATL08_30m_20190818082057_07830402_003_01_30m.csv', '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/11/17/197105/ATL08_30m_20190822081238_08440402_003_01_30m.csv']
atl08_tmp = pd.concat([pd.read_csv(f) for f in all_atl08_csvs_FOUND ], sort=False, ignore_index=True)
in_bounds = [-117.10749852280769, -116.50936927974429, 50.78795362739066, 51.16389512140189]
atl08_tmp = atl08_tmp[(atl08_tmp["lat"] > in_bounds[2]) &
                   (atl08_tmp["lat"] < in_bounds[3]) &
                   (atl08_tmp["lon"] > in_bounds[0]) &
                   (atl08_tmp["lon"] < in_bounds[1])
                               ]
print(len(atl08_tmp))
atl08_tmp.info()